In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os

In [6]:
# Prepare paths of input images and target segmentation masks
left_input_dir = "./output_images/output/left/"
right_input_dir = "./output_images/output/right/"
depth_dir = "./output_images/depth"

img_size = (320, 320)          #This is the biggest image I could get
num_classes = 4
batch_size = 16                # Bigger sizes sometimes couse crashes due to low memory
lower_color_threshold = 100


# Sorting image names in left input folder
left_input_img_paths = sorted([
    os.path.join(left_input_dir, fname)
    for fname in os.listdir(left_input_dir)
    if fname.endswith(".jpg")
])

# Sorting image names in left input folder
right_input_img_paths = sorted([
    os.path.join(right_input_dir, fname)
    for fname in os.listdir(right_input_dir)
    if fname.endswith(".jpg")
])

# Sorting image names in target folder
target_img_paths = sorted([
    os.path.join(depth_dir, fname)
    for fname in os.listdir(depth_dir)
    if fname.endswith(".jpg")
])

print("Number of left input samples:", len(left_input_img_paths))
print("Number of right input samples:", len(right_input_img_paths))
print("Number of target samples:", len(target_img_paths))

#Input and target images are now corresponding
for input_path, target_path in zip(left_input_img_paths[:10], target_img_paths[:10]):
    print(input_path, "|", target_path)

Number of left input samples: 1119
Number of right input samples: 1119
Number of target samples: 1120
./output_images/output/left/image_depth_1701624237_257517568.jpg | ./output_images/depth/image_depth_1701624237_257517568.jpg
./output_images/output/left/image_depth_1701624238_197022720.jpg | ./output_images/depth/image_depth_1701624238_197022720.jpg
./output_images/output/left/image_depth_1701624238_857298688.jpg | ./output_images/depth/image_depth_1701624238_857298688.jpg
./output_images/output/left/image_depth_1701624239_787462144.jpg | ./output_images/depth/image_depth_1701624239_787462144.jpg
./output_images/output/left/image_depth_1701624240_603289856.jpg | ./output_images/depth/image_depth_1701624240_603289856.jpg
./output_images/output/left/image_depth_1701624241_240535808.jpg | ./output_images/depth/image_depth_1701624241_240535808.jpg
./output_images/output/left/image_depth_1701624241_954830080.jpg | ./output_images/depth/image_depth_1701624241_954830080.jpg
./output_images/

In [ ]:
def load_image(path, size):
    img = cv2.imread(path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, size)
    return img

fig, axes = plt.subplots(1, 3, figsize=(40, 20))

index = 100
size= (640, 360)

# Assuming there are exactly three images
axes[0].imshow(load_image(left_input_img_paths[index], size))
axes[0].set_title("Left image")
axes[0].axis('off')

axes[1].imshow(load_image(right_input_img_paths[index], size))
axes[1].set_title("Right image")
axes[1].axis('off')

axes[2].imshow(load_image(target_img_paths[index], size))
axes[2].set_title("Depth image")
axes[2].axis('off')

plt.show()